In [2]:
import random
import re
import pickle

import openai
openai.api_key = 'Fill'

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.l2 import Ether, ARP
from scapy.layers.inet import IP, TCP, ICMP

openai.__version__

C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


'1.10.0'

In [3]:
packets_per_request = 5 #Paquetes que pedir al modelo de IA

In [6]:
with open("../data/text/tcp_summaries.txt","r") as f:
    packets_summary = f.read().splitlines()
len(packets_summary)

FileNotFoundError: [Errno 2] No such file or directory: '../data/text/tcp_summaries.txt'

In [41]:
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

In [43]:
responses = pickle.load( open("../data/pickle/davinci_pairs_Aday.pkl", "rb" ))


In [45]:
responses = []
len(responses)

0

In [47]:
# Defines que protocolo estás probando
protocol = 'TCP'

In [49]:
for i in range(2):
    
    print("...........................................")
    print( " GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter " +str(i+1) +")")
    
    # Le pones unos datos preseleccionados para que sepa como crear cada uno de los paquetes, en este caso, son un paquete ICMP y otro DNS
    if protocol == 'ICMP-DNS':
        query = "This is a packet summary:\n"
        query += "192.168.1.194 → 192.168.1.190 ICMP 100 Echo (ping) request id=0x9015, seq=18917\n"
        query += "This is the python code to generate the packet:\n"
        query += 'scapy.IP(src="192.168.1.194", dst="192.168.1.190")/scapy.ICMP(type=8, id=0x9015, seq=18917)\n\n'
        query += "This is a packet summary:\n"
        query += "192.168.1.190 → 192.168.1.152 DNS 193 Standard query response 0x0971 No such name AAAA a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com SOA ns-1926.awsdns-48.co.uk\n"
        query += "This is the python code to generate the packet:\n"
        query += 'IP(src="192.168.1.190", dst="192.168.1.152")/DNS(id=0x0971, qr=1, opcode=0, rcode=3, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com", qtype="SOA", qclass="IN"))\n\n'
    
    elif protocol == 'ARP':
        query = "This is a packet summary:\n"
        query += "c2:3d:19:6c:00:01     c2:3c:19:6c:00:01     ARP      60     Who has 10.0.0.2? Tell 10.0.0.1\n"
        query += "This is the python code to generate the packet:\n"
        query += 'scapy.Ether(src="c2:3d:19:6c:00:01", dst="c2:3c:19:6c:00:01")/scapy.ARP(op=1, pdst="10.0.0.2", psrc="10.0.0.1", hwdst="c2:3c:19:6c:00:01", hwsrc="c2:3d:19:6c:00:01")\n\n'
        query += "This is a packet summary:\n"
        query += "c2:3c:19:6c:00:01     c2:3d:19:6c:00:01     ARP      60     10.0.0.2 is at c2:3c:19:6c:00:01\n"
        query += "This is the python code to generate the packet:\n"
        query += 'scapy.Ether(src="c2:3c:19:6c:00:01", dst="c2:3d:19:6c:00:01")/scapy.ARP(op=2, psrc="10.0.0.2", hwsrc="c2:3c:19:6c:00:01", hwdst="c2:3d:19:6c:00:01", )\n\n'
    
    elif protocol == 'TCP':
        query = "This is a TCP packet summary in IPv4:\n"
        query += '193.24.227.230        9.9.9.9               TCP      64     40301 → 53 [ACK] Seq=45 Ack=108 Win=29312 Len=0\n'
        query += "This is the python code to generate the packet:\n"
        query += 'scapy.IP(dst="9.9.9.9", src="193.24.227.230")/scapy.TCP(sport=40301, dport=53, flags="A", seq=45, ack=108, window=29312)\n\n'
        query = "This is a TCP packet summary in IPv6:\n"
        query += '2001:470:765b::b15:22 2620:fe::fe           TCP      78     42982 → 53 [ACK] Seq=1 Ack=1 Win=28800 Len=0\n'
        query += "This is the python code to generate the packet:\n"
        query += 'scapy.IPv6(dst="2620:fe::fe", src="2001:470:765b::b15:22")/TCP(sport=42982, dport=53, flags="A", seq=1, ack=1, window=28800)\n\n'
    
    ## Luego seleccionas aleatoriamente resumenes ya generados para pedirle que te saque el comando a partir del paquete
    query += "These are the packet summaries:\n"
    summaries = random.choices(packets_summary,k=packets_per_request)
    text_sum = ""
    for summary in summaries:
        query += summary + "\n"
        text_sum += summary + "\n"

    # Le pides que te genere los paquetes con scapy y los ponga en una lista llamada pkt_list
    query += "\nGenerate python code for creating these packets with scapy framework and put them all in a list named pkt_list."

    #Query es el mensaje que le envias al modelo
    print(query)
    
    #Esta es la petición que le mandas al modelo de IA
    completion = openai.completions.create(
        model="gpt-3.5-turbo-instruct", 
        prompt=query,
        max_tokens=2600,
        temperature=0.1)
    
    print("...Generated!")
    
    # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
    responses.append(response(text_sum, completion))
    
    # Guardamos tanto la petición como la respuesta en un archivo
    pickle.dump(responses, open( "../data/pickle/davinci_pairs_Aday.pkl", "wb" ) )
    
    print("Number of completitions done: " + str(len(responses)))
    

...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 1)
This is a TCP packet summary in IPv6:
2001:470:765b::b15:22 2620:fe::fe           TCP      78     42982 → 53 [ACK] Seq=1 Ack=1 Win=28800 Len=0
This is the python code to generate the packet:
scapy.IPv6(dst="2620:fe::fe", src="2001:470:765b::b15:22")/TCP(sport=42982, dport=53, flags="A", seq=1, ack=1, window=28800)

These are the packet summaries:
192.168.7.15          192.168.7.17          TCP      66     36504 â†’ 10051 [FIN, ACK] Seq=89 Ack=105 Win=64256 Len=0 TSval=2239358533 TSecr=2250046823
2a02:26f0:6c00::210:ba48 2001:470:765b:0:1c6e:18ae:ddb4:3bc1 TCP      74     80 â†’ 44718 [ACK] Seq=1 Ack=203 Win=29952 Len=0
2a00:6020:5004:26aa:ccdd:5e3b:815:bcd5 2a01:238:4363:ee00:9169:a8a4:e572:d5f8 TCP      86     [TCP Retransmission] 11189 â†’ 21 [SYN] Seq=0 Win=64800 Len=0 MSS=1440 WS=256 SACK_PERM
fe80::3a63:bbff:fe02:5a4d fe80::ca4:b90c:5dad:e86b TCP      1418   631 â†’ 56775 [ACK

### Variables Importantes en completions de OpenAI

1. **temperature**: What sampling temperature to use, between 0 and 2. Higher values like 0.8 will
                make the output more random, while lower values like 0.2 will make it more
                focused and deterministic.
              
2. **timeout**: Override the client-level default timeout for this request, in seconds

3. **n**: How many completions to generate for each prompt.

4. **max_tokens**: The maximum number of [tokens](/tokenizer) that can be generated in the
              completion.

              The token count of your prompt plus `max_tokens` cannot exceed the model's
              context length.
              [Example Python code](https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken)
              for counting tokens.
 
5. **stop**: Up to 4 sequences where the API will stop generating further tokens. The
              returned text will not contain the stop sequence.
 
6. **best_of**: Generates `best_of` completions server-side and returns the "best" (the one with
              the highest log probability per token). Results cannot be streamed.

              When used with `n`, `best_of` controls the number of candidate completions and
              `n` specifies how many to return – `best_of` must be greater than `n`.

              **Note:** Because this parameter generates many completions, it can quickly
              consume your token quota. Use carefully and ensure that you have reasonable
              settings for `max_tokens` and `stop`.
              
7. **prompt**: The prompt(s) to generate completions for, encoded as a string, array of
              strings, array of tokens, or array of token arrays.

              Note that <|endoftext|> is the document separator that the model sees during
              training, so if a prompt is not specified the model will generate as if from the
              beginning of a new document.
              
              
MUCHOS MÁS en https://github.com/openai/openai-python/blob/0c1e58d511bd60c4dd47ea8a8c0820dc2d013d1d/src/openai/resources/completions.py#L35

In [52]:
#Esta es la forma que tiene una respuesta del modelo de IA
print(responses[-1].completion)

Completion(id='cmpl-90okuDJUN5hYn147RXjBI3Zc8VEhH', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='\n\npkt1 = scapy.IP(dst="2a01:cb0c:8315:a400:2118:60c1:e8e8:35b0", src="2001:470:765b:0:1c6e:18ae:ddb4:3bc1")/TCP(sport=80, dport=55031, flags="A", seq=1421, ack=172, window=29568, options=[("MSS", 1460)])/Raw(load="Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed euismod, nisi eu ultricies tristique, velit urna aliquet sapien, vel tincidunt nunc est vel nunc.")\npkt2 = scapy.IPv6(dst="2620:fe::fe", src="2001:470:765b::b15:22")/TCP(sport=53, dport=47038, flags="A", seq=1, ack=39, window=28928)\npkt3 = scapy.IPv6(dst="fe80::ca4:b90c:5dad:e86b", src="fe80::3a63:bbff:fe02:5a4d")/TCP(sport=631, dport=56775, flags="A", seq=620298, ack=394, window=8847, options=[("NOP", None), ("NOP", None), ("Timestamp", (3848298, 406434051))])/Raw(load="Suspendisse potenti. Fusce euismod, nisi eu ultricies tristique, velit urna aliquet sapien, vel tincidunt nunc est 

In [54]:
# Este es el resumen de lo que está en el struct de responses para la última petición
print(responses[-1].prompt_summary)
print(responses[-1].completion.choices[0].text)

2001:470:765b:0:1c6e:18ae:ddb4:3bc1 2a01:cb0c:8315:a400:2118:60c1:e8e8:35b0 TCP      1514   80 â†’ 55031 [ACK] Seq=1421 Ack=172 Win=29568 Len=1420 [TCP segment of a reassembled PDU]
2620:fe::fe           2001:470:765b::b15:22 TCP      78     53 â†’ 47038 [ACK] Seq=1 Ack=39 Win=28928 Len=0
fe80::3a63:bbff:fe02:5a4d fe80::ca4:b90c:5dad:e86b TCP      1418   631 â†’ 56775 [ACK] Seq=620298 Ack=394 Win=8847 Len=1320 TSval=3848298 TSecr=406434051 [TCP segment of a reassembled PDU]
10.200.200.102        10.200.200.101        TCP      72     [TCP Retransmission] 46330 â†’ 646 [ACK] Seq=1 Ack=19 Win=4110 Len=18
192.168.7.12          192.168.7.1           TCP      64     1227 â†’ 51108 [ACK] Seq=1761 Ack=158 Win=65378 Len=0



pkt1 = scapy.IP(dst="2a01:cb0c:8315:a400:2118:60c1:e8e8:35b0", src="2001:470:765b:0:1c6e:18ae:ddb4:3bc1")/TCP(sport=80, dport=55031, flags="A", seq=1421, ack=172, window=29568, options=[("MSS", 1460)])/Raw(load="Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed e

### Generación de paquetes

In [57]:
# Ejecutamos los comandos que devuelve el modelo para generar los paquetes
exec(responses[-1].completion.choices[0].text)
print(pkt_list)

gaierror: [Errno 11001] getaddrinfo failed

In [59]:
send(pkt_list)

NameError: name 'pkt_list' is not defined

## Introducción en un PCAP

In [62]:
# Guardamos los paquetes generados en un pcap
with open("../data/pcap/createdChatGPT/ARPgenerated_3_5_turbo-instruct_commands.pcap", "wb") as f:
    wrpcap(f, pkt_list)

NameError: name 'pkt_list' is not defined